# Eat Safe, Love

## Notebook Set Up

In [29]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [30]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [31]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [32]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [33]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [34]:
# Find the establishments with a hygiene score of 20
hygiene20 = establishments.find({'scores.Hygiene': {'$eq':20}})

# Use count_documents to display the number of documents in the result
hygiene_count = establishments.count_documents({'scores.Hygiene': {'$eq':20}})
print(f"The number of establishments with a hygiene score of 20 are: {hygiene_count}")
print("This is the first document:")

# Display the first document in the results using pprint
pprint(establishments.find_one({'scores.Hygiene': {'$eq':20}}))

The number of establishments with a hygiene score of 20 are: 41
This is the first document:
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('663c038314ad94276425139b'),
 'geocode': {'latitude': Decimal128('50.769705'),
             'longitude': Decimal128('0.27694')},
 'links': [{'href': 'http

In [35]:
# Convert the result to a Pandas DataFrame
hygiene_df = pd.DataFrame(hygiene20)
hygiene_df[['longitude','latitude']] = pd.json_normalize(hygiene_df['geocode'])
hy

# Display the number of rows in the DataFrame
# print("Rows in DataFrame: ", len(hygiene_df))

# Display the first 10 rows of the DataFrame
#test_df.head(10)
hygiene_df.head()

_id  FHRSID  ChangesByServerID  \
0  663c038314ad94276425139b  110681                  0   
1  663c038314ad942764251717  612039                  0   
2  663c038314ad942764251a1f  730933                  0   
3  663c038314ad942764251c19  172735                  0   
4  663c038314ad942764251c23  172953                  0   

  LocalAuthorityBusinessID         BusinessName  \
0                     4029  The Chase Rest Home   
1                1970/FOOD           Brenalwood   
2                1698/FOOD        Melrose Hotel   
3             PI/000023858        Seaford Pizza   
4             PI/000024532        Golden Palace   

                        BusinessType  BusinessTypeID           AddressLine1  \
0                    Caring Premises               5   5-6 Southfields Road   
1                    Caring Premises               5              Hall Lane   
2  Hotel/bed & breakfast/guest house            7842  53 Marine Parade East   
3             Takeaway/sandwich shop            7844          4 High Street   
4            Restaurant/Cafe/Canteen               1         5 South Street   

         AddressLine2 AddressLine3  ...  \
0          Eastbourne  East Sussex  ...   
1  Walton-on-the-Naze        Essex  ...   
2      Clacton On Sea        Essex  ...   
3             Seaford  East Sussex  ...   
4             Seaford  East Sussex  ...   

                                              scores SchemeType  \
0  {'Hygiene': 20, 'Structural': 20, 'ConfidenceI...       FHRS   
1  {'Hygiene': 20, 'Structural': 15, 'ConfidenceI...       FHRS   
2  {'Hygiene': 20, 'Structural': 20, 'ConfidenceI...       FHRS   
3  {'Hygiene': 20, 'Structural': 10, 'ConfidenceI...       FHRS   
4  {'Hygiene': 20, 'Structural': 10, 'ConfidenceI...       FHRS   

                                          geocode  RightToReply     Distance  \
0   {'longitude': 0.27694, 'latitude': 50.769705}                4613.888288   
1  {'longitude': 1.278721, 'latitude': 51.857536}                4617.965824   
2   {'longitude': 1.15927, 'latitude': 51.789429}                4619.656144   
3   {'longitude': 0.10202, 'latitude': 50.770885}                4620.421725   
4  {'longitude': 0.101446, 'latitude': 50.770724}                4620.437179   

  NewRatingPending                                               meta  \
0            False  {'dataSource': None, 'extractDate': '0001-01-0...   
1            False  {'dataSource': None, 'extractDate': '0001-01-0...   
2            False  {'dataSource': None, 'extractDate': '0001-01-0...   
3            False  {'dataSource': None, 'extractDate': '0001-01-0...   
4            False  {'dataSource': None, 'extractDate': '0001-01-0...   

                                               links longitude   latitude  
0  [{'rel': 'self', 'href': 'https://api.ratings....   0.27694  50.769705  
1  [{'rel': 'self', 'href': 'https://api.ratings....  1.278721  51.857536  
2  [{'rel': 'self', 'href': 'https://api.ratings....   1.15927  51.789429  
3  [{'rel': 'self', 'href': 'https://api.ratings....   0.10202  50.770885  
4  [{'rel': 'self', 'href': 'https://api.ratings....  0.101446  50.770724  

[5 rows x 30 columns]

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [36]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.


match_query = {'$match': {'LocalAuthorityName': {'$regex': 'London'}}}
group_query = {'$group': {'_id': '$RatingValue',
                          'RatingValue': {'$gte':4}}}

pipeline = [match_query, group_query]

results = list(establishments.aggregate(pipeline))

print("Number of establishments in result: ", len(results))

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


OperationFailure: unknown group operator '$gte', full error: {'ok': 0.0, 'errmsg': "unknown group operator '$gte'", 'code': 15952, 'codeName': 'Location15952'}

In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude =
longitude =

query =
sort =
limit =

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
